In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('data/Train.csv')

In [4]:
df.head(5)

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price
0,0,2011-01-01,531,NaN,12,35,1,1,25.0,NaN,...,0,0,0,0,0,0,0,0,0,3721000
1,1,2011-01-01,385,2.0,15,46,1,2,15.0,1.0,...,0,0,0,0,0,0,0,0,0,7431000
2,2,2011-01-01,51,0.0,1,53,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,3081000
3,3,2011-01-01,76,0.0,1,60,3,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,3847000
4,4,2011-01-01,114,0.0,2,43,2,0,30.0,0.0,...,0,0,0,0,0,0,0,0,0,2323000


In [5]:
df['year'] = df['date'].map(lambda x: int(str(x).split('-')[0]) )

In [6]:
coeff_x=1.107

In [7]:
df['month'] = df['date'].map(lambda x: int(str(x).split('-')[1]) )

In [8]:
df['metro_dist'] = df['metro_dist'].fillna(
                                        df.metro_dist.mean()
                                        )

## Бинаризация


In [53]:
def cat_to_num(data):
    categories = pd.unique(data)
    features = {}
    for cat in categories:
        binary = (data == cat)
        features['street_'+str(cat)] = binary.astype('int')
    return pd.DataFrame(features)

df = df.join(cat_to_num(df['street_id']))    

In [54]:
df_extemes = df[df.area>90]
df = df[df.area <= 90]


In [13]:
df['price'] = df.apply(lambda x: x['price']*coeff if x['year']==2011 else x['price'], axis=1)

In [18]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [19]:
df[df.build_tech>=0].count()

id            67936
date          67936
street_id     67936
build_tech    67936
floor         67936
area          67936
rooms         67936
balcon        67936
metro_dist    67936
g_lift        47395
n_photos      67936
kw1           67936
kw2           67936
kw3           67936
kw4           67936
kw5           67936
kw6           67936
kw7           67936
kw8           67936
kw9           67936
kw10          67936
kw11          67936
kw12          67936
kw13          67936
price         67936
year          67936
street_531    67936
street_385    67936
street_51     67936
street_76     67936
              ...  
street_485    67936
street_523    67936
street_88     67936
street_67     67936
street_453    67936
street_376    67936
street_226    67936
street_261    67936
street_456    67936
street_86     67936
street_292    67936
street_375    67936
street_583    67936
street_419    67936
street_427    67936
street_610    67936
street_246    67936
street_568    67936
street_397    67936


In [100]:
train, test = train_test_split(df, test_size = 0.6)

In [31]:
features = list(set(df.columns) - set(['id', 'price', 'street_id', 'year', 'g_lift', 'date']))

In [101]:
lgb_data_train = lgb.Dataset(train[features].values, train['price'])
lgb_data_test = lgb.Dataset(test[features].values, test['price'])

In [91]:
params = {
    'objective':'regression', 
    'metric': 'mae',
    'learning_rate': 0.07, 
    'nthread': 4,
    'reg_alpha': 45
}

In [102]:
model2 = lgb.train(params, lgb_data_train, 
                  valid_sets=[lgb_data_train, lgb_data_test],
                  num_boost_round=5000,
                  early_stopping_rounds=100, verbose_eval=40)

Training until validation scores don't improve for 100 rounds
[40]	training's l1: 1.431e+06	valid_1's l1: 1.451e+06
[80]	training's l1: 1.21312e+06	valid_1's l1: 1.24333e+06
[120]	training's l1: 1.07014e+06	valid_1's l1: 1.105e+06
[160]	training's l1: 975410	valid_1's l1: 1.01344e+06
[200]	training's l1: 910155	valid_1's l1: 951342
[240]	training's l1: 863091	valid_1's l1: 906903
[280]	training's l1: 826982	valid_1's l1: 873084
[320]	training's l1: 798333	valid_1's l1: 846788
[360]	training's l1: 775757	valid_1's l1: 827175
[400]	training's l1: 756711	valid_1's l1: 810768
[440]	training's l1: 740050	valid_1's l1: 796512
[480]	training's l1: 726449	valid_1's l1: 785245
[520]	training's l1: 714274	valid_1's l1: 775474
[560]	training's l1: 703625	valid_1's l1: 767352
[600]	training's l1: 694200	valid_1's l1: 760929
[640]	training's l1: 685953	valid_1's l1: 755456
[680]	training's l1: 677775	valid_1's l1: 750158
[720]	training's l1: 671323	valid_1's l1: 746046
[760]	training's l1: 665478	v

# Выбросы

In [39]:
import sklearn.ensemble

In [55]:
df_extemes

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,street_62,street_416,street_462,street_326,street_512,street_178,street_407,street_495,street_349,street_560
14,14,2011-01-01,384,2.0,6,99,3,2,0.000000,1.0,...,0,0,0,0,0,0,0,0,0,0
16,16,2011-01-01,75,NaN,11,93,3,0,5.000000,NaN,...,0,0,0,0,0,0,0,0,0,0
47,47,2011-01-01,9,NaN,5,185,5,1,5.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
98,98,2011-01-01,59,1.0,9,97,3,1,5.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
137,137,2011-01-01,231,1.0,9,183,5,1,30.000000,1.0,...,0,0,0,0,0,0,0,0,0,0
144,144,2011-01-01,517,1.0,12,99,3,1,20.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
167,167,2011-01-01,647,2.0,12,102,3,2,30.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
187,187,2011-01-01,581,2.0,9,166,4,0,10.000000,1.0,...,0,0,0,0,0,0,0,0,0,0
260,260,2011-01-01,595,1.0,1,95,3,1,21.858543,NaN,...,0,0,0,0,0,0,0,0,0,0
373,373,2011-01-01,230,1.0,12,183,5,1,30.000000,NaN,...,0,0,0,0,0,0,0,0,0,0


In [106]:
model_extremes = sklearn.ensemble.RandomForestRegressor(n_jobs=4, n_estimators=100)

In [121]:
model_extremes.fit(df_extemes.drop(['price','date','year', 'build_tech',  'g_lift'], axis=1) ,
                   df_extemes['price'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [58]:
from sklearn.metrics import mean_absolute_error as MAE

In [122]:
MAE( df_extemes['price'], 
    model_extremes.predict(df_extemes.drop(['price','date','build_tech','year', 'g_lift'], axis=1))  )

1580345.4561342187

# Предсказание

In [62]:
df_pred = pd.read_csv('data/Test.csv')

In [63]:
df_pred['metro_dist'] = df_pred['metro_dist'].fillna(
                                        df_pred.metro_dist.mean()
                                        )

In [64]:
df_pred['year'] = df_pred['date'].map(lambda x: int(str(x).split('-')[0]) )

In [107]:
df_pred['month'] = df_pred['date'].map(lambda x: int(str(x).split('-')[1]) )

In [65]:
def cat_to_num(data):
    categories = pd.unique(data)
    features = {}
    for cat in categories:
        binary = (data == cat)
        features['street_'+str(cat)] = binary.astype('int')
    return pd.DataFrame(features)

df_pred = df_pred.join(cat_to_num(df_pred['street_id']))   

In [103]:
df_pred['price_norm'] = model.predict(df_pred[features])

In [104]:
df_pred['price_norm1'] = model1.predict(df_pred[features])

In [105]:
df_pred['price_norm2'] = model2.predict(df_pred[features])

In [123]:
df_pred['price_x_new'] = model_extremes.predict(df_pred.drop(['date','build_tech','year','g_lift', 'price_norm', 'price_norm1', 'price_norm2', 'price','price_x'], axis=1))

In [126]:
df_pred

,id,date,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,...,street_549,street_367,street_476,street_484,price_norm,price_x,price,price_norm1,price_norm2,price_x_new
0,100000,2012-06-01,427,NaN,9,60,3,1,30.000000,0.0,...,0,0,0,0,4.375467e+06,1.292924e+07,4319765,4.544053e+06,4.117755e+06,13781190.0
1,100001,2012-06-01,593,1.0,7,52,2,1,20.000000,0.0,...,0,0,0,0,9.480781e+06,1.673336e+07,9671770,1.018309e+07,9.493561e+06,15788650.0
2,100002,2012-06-01,91,0.0,4,58,3,0,30.000000,NaN,...,0,0,0,0,2.803662e+06,1.070637e+07,2945342,2.839899e+06,2.841441e+06,9957280.0
3,100003,2012-06-01,224,1.0,8,60,3,1,30.000000,0.0,...,0,0,0,0,8.159099e+06,1.087169e+07,7611609,7.331364e+06,7.452784e+06,12056190.0
4,100004,2012-06-01,426,NaN,2,54,3,0,30.000000,1.0,...,0,0,0,0,1.185160e+06,1.112388e+07,1223867,1.229279e+06,1.124272e+06,12316890.0
5,100005,2012-07-01,1,NaN,9,34,1,0,30.000000,0.0,...,0,0,0,0,1.622736e+06,9.944400e+06,1656981,1.655913e+06,1.673588e+06,10086940.0
6,100006,2012-07-01,203,0.0,3,58,3,0,21.940492,0.0,...,0,0,0,0,6.294244e+06,1.263784e+07,6100889,5.536591e+06,6.055409e+06,11606260.0
7,100007,2012-07-01,6,NaN,4,41,2,0,30.000000,1.0,...,0,0,0,0,3.350518e+06,9.922667e+06,3311867,3.183517e+06,3.477284e+06,10542630.0
8,100008,2012-07-01,267,0.0,12,63,3,0,25.000000,1.0,...,0,0,0,0,2.452367e+06,1.371472e+07,2442062,2.426748e+06,2.473710e+06,12221850.0
9,100009,2012-07-01,135,1.0,3,52,2,1,20.000000,1.0,...,0,0,0,0,4.338471e+06,1.233772e+07,4549418,4.672580e+06,4.559249e+06,15369160.0


In [125]:
def f(x):
    if (x['area'] > 90) :
        if (x['year'] == 2012): return int( (x['price_x']+x['price_x_new'])/2.0)
        else: return int( (x['price_x']+x['price_x_new'])*coeff_x/2.0)
    else:
        if (x['year'] == 2012): return int( (x['price'] + x['price_norm'] + x['price_norm1'] + x['price_norm2'])/4.0 )
        else: return int( (x['price'] + (x['price_norm'] + x['price_norm1'] + x['price_norm2'])*coeff )/4.0 )
        
df_pred['price'] = df_pred.apply(lambda x: f(x), axis=1)

In [127]:
df_pred[["id", "price"]].to_csv("y_pred3.csv", index=False)

!cat y_pred3.csv | head

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
